    这段代码主要用于设置一个深度学习模型的基础参数，包括天线数量、训练参数、评估参数以及SB算法的特定参数。

In [ ]:
# 导入必要的库
import math  # 提供基本的数学运算
import random  # 用于生成随机数
import numpy as np  # 用于数值计算
import copy  # 深拷贝对象
import matplotlib.pyplot as plt  # 用于数据可视化
import torch  # 导入PyTorch库，用于深度学习
import torch.nn as nn  # 导入PyTorch的神经网络模块
import torch.optim as optim  # 导入PyTorch的优化器模块
import torch.nn.functional as F  # 导入PyTorch的功能性模块，包含多种神经网络功能
from scipy.linalg import toeplitz  # 从SciPy库导入toeplitz函数，用于生成Toeplitz矩阵
import logging

## 配置日志记录器

# 创建日志记录器
logger = logging.getLogger('my_logger')
logger.setLevel(logging.DEBUG)
# 创建文件处理器
file_handler = logging.FileHandler('24_12_01.log')
file_handler.setLevel(logging.DEBUG)
# 创建控制台处理器
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
# 创建日志格式化器
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# 将格式化器添加到处理器
file_handler.setFormatter(formatter)
console_handler.setFormatter(formatter)
# 将处理器添加到日志记录器
logger.addHandler(file_handler)
logger.addHandler(console_handler)

# 设置计算设备
device = torch.device('cuda') # 'cpu' or 'cuda'
print(torch.__version__)  # 打印PyTorch的版本


# 模型参数
n=16 # 发射天线数量
m=16 # 接收天线数量
##
N=2*n  # 计算总发射天线数量
M=2*m  # 计算总接收天线数量

# 训练参数
max_itr = 10 # 最大迭代次数
bs = 2000 # 每个小批量的样本大小，即每次训练使用的样本数量
num_batch = 1000 # 小批量的数量，即总的训练批次
lr_adam = 5e-3 # Adam 优化器的学习率
##

# 评估泛化误差的参数
#total_itr=30 # total number of iterations (multiple number of "itr")  ## 总迭代次数（多个“itr”的乘积）
sample = 1000  # 评估时的样本数量
bs_ = 2000  # 评估时的小批量大小
##


# SB 设置
eps = 1.0 # 控制某种阈值或精度的参数，通常用于算法收敛
T_max = max_itr # 最大迭代次数，与 max_itr 相同
pump_SB = 1.0/(T_max*eps) # 泵系数，计算公式为 1.0 / (T_max * eps)
D_SB = 1. # delta参数，通常用于控制学习率或收敛速度（步长参数）
xi_SB = 0.1 # 初始xi值，可能用于某种动态调整（停止条件或更新策略）

2.0.1


这段代码定义了一个基于深度展开技术的信号检测器，用于处理无线通信中的信号恢复问题。

In [4]:
# 生成随机输入信号x
def x_gen(bs,n):  # 参数 bs(int):批量大小 n(int):天线数量 返回Tensor: 生成的信号x
    # torch.rand(bs,n) 会创建一个形状为 (bs, n) 的张量（即一个包含 bs 行和 n 列的矩阵），其中的元素是从标准均匀分布（在区间 [0, 1) 上）随机采样的浮点数。
    # .to(device) 是一个方法，它将生成的张量 x 转移到指定的设备上。这里的 device 可以是CPU或GPU。
    # 如果 device 是一个GPU设备，这个操作会将张量 x 从CPU内存复制到GPU内存中，以便后续的计算可以在GPU上执行，从而利用GPU的并行处理能力加速计算。
    x = torch.rand(bs,n).to(device)  
    x[x<0.5] = -1  # 将小于0.5的值设为-1
    x[x>0.5] = 1  # 将大于0.5的值设为1
    return x

# 生成带有噪声的接收信号y
# 参数 bs (int): 批量大小 m (int): 接收天线数量  x0 (Tensor): 发送信号 
# H (Tensor): 信道矩阵 sigma_std (float): 噪声标准差
# 返回Tensor: 生成的接收信号y
def y_gen(bs,m,x0,H,sigma_std):  
    # 模拟一个信号通过一个信道的过程，并添加高斯噪声
    return x0@H+ torch.normal(0.0, sigma_std*torch.ones(bs, m)).to(device)
    # x0 是一个张量，代表原始信号。
    # H 是信道矩阵，代表信号传输过程中的信道效应。
    # @ 是PyTorch中的矩阵乘法运算符。
    # sigma_std 是根据信噪比（SNR）计算得到的标准差，它控制噪声的强度。
    # torch.ones(bs, m) 生成一个形状为 (bs, m) 的张量，其中的元素都是 1。
    # sigma_std * torch.ones(bs, m) 将每个元素的标准差乘以 sigma_std，生成一个标准差与信噪比相关的噪声张量。

# 将H和y转换成QUBO形式
def trans_2_QUBO(H,y):
    J = H@H.t() - torch.diag(torch.diagonal(H@H.t(),0))  # 构建二次项的耦合强度矩阵 J
    # H.t() 是 H 的转置。
    # torch.diagonal(H @ H.t(), 0) 从 H @ H.t() 的结果中提取主对角线上的元素（即 diagonal=0 表示主对角线）。
    # torch.diag(torch.diagonal(H @ H.t(), 0)) 将提取的对角元素重新构造成一个对角矩阵。
    h = -2*y@H.t()  # 计算线性项
    # y @ H.t() 计算观测信号 y 和信道矩阵转置 H.t() 的矩阵乘积，结果是一个向量。
    # 这个缩放因子 -2 是构建QUBO问题时的常见选择，用于调整目标函数中线性项的系数。
    # 结果 h 是一个向量，其中的每个元素是观测信号 y 通过信道矩阵 H 后的线性变换结果，乘以 -2 后得到的偏置项系数。
    lmax_2 = ((J*J).sum()/(N*(N-1)))**0.5 # 估计耦合强度矩阵 J 的最大特征值
    # sum() 函数计算矩阵 J * J 中所有元素的总和。
    # N 是变量的数量，即矩阵 J 的大小。N * (N - 1) 是矩阵 J 中非对角元素的总数。将总和除以 N * (N - 1) 进行归一化，得到 J 矩阵元素平方的平均值。
    # 开平方根 ** 0.5
    return J,h, 1.0/(2*N**0.5*lmax_2)  # 返回三个值：J，h，以及一个归一化因子。
    # 2 * N ** 0.5 是一个缩放因子，它考虑了变量数量的平方根。有助于平衡目标函数中的不同项。

# LMMSE下的QUBO转换
def trans_2_QUBO_LMMSE(H,y,lam):  # 参数 lam (Tensor): 正则化参数
    H_inv = torch.linalg.inv(H.t()@H+lam*torch.eye(M,device=device)) # 计算矩阵H^TH+λI的逆
    # torch.eye(M, device=device) 创建一个大小为M*M的单位矩阵，其中M是H的列数。
    # lam 是正则化参数，用于控制正则化项的强度。
    # torch.linalg.inv() 计算上述矩阵的逆。
    J = H@H_inv@H.t() - torch.diag(torch.diagonal(H@H_inv@H.t(),0))
    h = -2*y@H_inv@H.t()
    lmax_2 = ((J*J).sum()/(N*(N-1)))**0.5 #estimated max. eig.
    return J,h, 1.0/(2*N**0.5*lmax_2)

def BER(x,y):
    z = torch.ones(x.size()).to(device)
    z[torch.isclose(torch.sign(x),torch.sign(y))] = 0.
    return z.sum()/(z.numel())

seed_ =12
torch.manual_seed(seed_)

# QPSK
def H_gen(m,n):
    H_re = torch.normal(0.0, std=math.sqrt(0.5) * torch.ones(n,m))
    H_im = torch.normal(0.0, std=math.sqrt(0.5) * torch.ones(n,m))  # sensing matrix
    H = torch.cat((torch.cat((H_re,H_im),0),torch.cat((-1*H_im,H_re),0)),1)
    H = H.to(device)
    return H


def est_SNR(snr,m,n):
    sigma2 = (2*n/math.pow(10,snr/10.0))/2.0
    sigma_std = math.sqrt(sigma2)
    return sigma_std

### DU
class DU_dSB(nn.Module):
    def __init__(self):
        super(DU_dSB,self).__init__()
        self.eps = nn.Parameter( eps + 0.01 * torch.randn(max_itr,device=device))
        self.lam = nn.Parameter(torch.ones(1,device=device))
        self.xi = nn.Parameter(torch.ones(1,device=device))
        self.beta = nn.Parameter(1.0*torch.ones(max_itr,device=device))

    def Pump(self,t):
        tsum = (self.eps).sum()
        return t/tsum

    def Softclamp(self,x):
        si = nn.SiLU()
        return si(10*(x+1))/10-si(10*(x-1))/10 -1

    def dSB(self,k, q,p,t,eps,J,h,D_SB,pump_SB,xi_SB):
        sq = q
        DE_QUBO = 0.5 * h + sq@J
        p_ = p - eps * ((D_SB-self.Pump(t)) * q + self.xi*xi_SB * DE_QUBO) #diff(q,Po)
        q_ = q + (eps * D_SB) * p_ #diff(p,K)
        q_2 = self.Softclamp(q_) #torch.clamp(q_, min=-1., max=1.)
        p_ = p_ - p_ * torch.sigmoid(100* (torch.abs(q_)-1.01) )
        # naively torch.heaviside(1-torch.abs(q_), torch.zeros(1,device=device)) but its derivative is unaveilabe
        return q_2,p_

    def forward(self, H,y,itr,bs):
        J, h, xi_SB= trans_2_QUBO_LMMSE(H,y,self.lam**2)
        q = torch.zeros(bs, N,device=device) # x
        p = torch.zeros(bs, N,device=device) # y
        q_traj = np.zeros([T_max, N]) # trajectory
        p_traj = np.zeros([T_max, N]) # trajectory

        p = 0.01*torch.randn(bs, N,device=device) # random init point

        t = 0.0

        for i in range(itr):
            k = i % max_itr
            t = t + self.eps[k]
            q, p = self.dSB(k,q,p,t,self.eps[k],J,h,D_SB,pump_SB,xi_SB)
            q_traj[i]=q[0,:].cpu().detach().numpy()
            p_traj[i]=p[0,:].cpu().detach().numpy()

        return q, p, q_traj, p_traj

def train(snr):
    #SNR
    sigma_std = est_SNR(snr, m,n)

    network = DU_dSB().to(device)  # generating an instance of TPG-detector
    opt = optim.Adam(network.parameters(), lr=lr_adam )  # setting for optimizer
    torch.autograd.set_detect_anomaly(True)

    torch.manual_seed(1)
    network.train()


    print("-------------------")
    print("snr=",snr)
    for i in range(num_batch):#num_batch):
        H = H_gen(m,n)
        sol = x_gen(bs,N)
        y = y_gen(bs,M,sol,H,sigma_std)

        opt.zero_grad()
        x_hat,_,q_traj ,_= network(H,y,max_itr,bs)

        loss = F.mse_loss(x_hat, sol) #squared_loss
        loss.backward()

        if i % 100 == 0:
            print('loss{0}:{1}'.format(i,loss.data), "BER:", BER(sol,x_hat.sign())) #print_loss
        opt.step()

    print("-----\n SNR:",snr.item())
    print("eps=",network.eps)
    print("beta=",network.beta)
    print("xi=",network.xi)
    print("lam=",network.lam**2)
    print("train done")
    return network

# Generalization Error Evaluation
def eval(network,snr):
    ber_= 0.0
    it = max_itr
    sigma_std = est_SNR(snr, m,n)

    for i in range(sample):
        H = H_gen(m,n)
        sol = x_gen(bs_,N)
        y = y_gen(bs_,M,sol,H,sigma_std)
        xx = torch.zeros(bs_,N,device=device)

        res = 100*torch.ones(bs_,N,device=device)
        x_hat,_,q_traj ,_= network(H,y, it,bs_)
        res_ = (y-x_hat.sign()@H).norm(dim=1).view(bs_,1).repeat(1,N).view(bs_,N) # OK
        xx[res_<res] = x_hat[res_<res]
        res[res_<res] = res_[res_<res]
        ber_ += BER(sol,xx.sign())

    ber_ = ber_/sample
    print("SNR:",snr.item(),"BER (generalization):",ber_.item())
    return ber_.item()


# main part
print("#_ ", "n=", n, "m=",m,"max_itr=", max_itr, "bs=",bs, "num_batch=", num_batch,"learning_rate=", lr_adam)

snr_values = torch.arange(5, 31, 5)
ber_results = []

for snr in snr_values:
    net = train(snr)
    ber = eval(net, snr)
    ber_results.append(ber)

# # 使用 matplotlib 绘制折线图
# plt.figure(figsize=(10, 6))
# plt.title('DU-LM-SB')
# plt.xlabel('SNR (dB)')
# plt.ylabel('BER')
# plt.ylim(1e-6, 1)
# plt.legend()
# plt.grid(True, which='both', linestyle='--', linewidth=0.5)
# plt.show()


#_  n= 16 m= 16 max_itr= 50 bs= 2000 num_batch= 1000 learning_rate= 0.005
-------------------
snr= tensor(5)
loss0:0.7003787755966187 BER: tensor(0.1760, device='cuda:0')
loss100:0.6888607740402222 BER: tensor(0.1738, device='cuda:0')
loss200:0.6930116415023804 BER: tensor(0.1750, device='cuda:0')
loss300:0.6507097482681274 BER: tensor(0.1646, device='cuda:0')
loss400:0.63661789894104 BER: tensor(0.1611, device='cuda:0')
loss500:0.6091437339782715 BER: tensor(0.1540, device='cuda:0')
loss600:0.6838785409927368 BER: tensor(0.1726, device='cuda:0')
loss700:0.6755576133728027 BER: tensor(0.1701, device='cuda:0')
loss800:0.6998711228370667 BER: tensor(0.1762, device='cuda:0')
loss900:0.6735630035400391 BER: tensor(0.1695, device='cuda:0')
-----
 SNR: 5
eps= Parameter containing:
tensor([1.2484, 1.2858, 0.9469, 1.1548, 0.9421, 0.7760, 1.2299, 0.8947, 1.0993,
        1.3356, 0.9182, 0.9377, 1.4067, 0.7225, 0.8292, 1.3221, 1.0235, 0.9419,
        1.4237, 1.1222, 0.8795, 1.2661, 1.1246, 1.0210